In [1]:
%load_ext sql
%sql sqlite:///nbastats.db

In [2]:
%%sql 
SELECT name FROM sqlite_master
WHERE type='table'

 * sqlite:///nbastats.db
Done.


name
advanced_stats
allgames_stats
allstar_games_stats
game_highs_stats
per_game_stats
salaries
totals_stats


In [3]:
%%sql
/*Find the highest salary earned by each player in a single season*/

SELECT Player, Season, MAX(CAST(REPLACE(Salary,'$','') AS DOUBLE)) as Highest_Salary_USD 
FROM salaries
GROUP BY Player
ORDER BY Highest_Salary_USD DESC;


 * sqlite:///nbastats.db
Done.


Player,Season,Highest_Salary_USD
Lebron James,2017-18,33285709.0
Michael Jordan,1997-98,33140000.0
Kobe Bryant,2013-14,30453805.0


In [4]:
%%sql
/*Find the lowest salary earned by each player in a single season*/

SELECT Player, Season, MIN(CAST(REPLACE(Salary,'$','') AS DOUBLE)) as Lowest_Salary_USD 
FROM salaries
GROUP BY Player
ORDER BY Lowest_Salary_USD DESC;

 * sqlite:///nbastats.db
Done.


Player,Season,Lowest_Salary_USD
Lebron James,2003-04,4018920.0
Kobe Bryant,1996-97,1015000.0
Michael Jordan,1984-85,550000.0


In [5]:
%%sql
/*Find the total salary earned by each player*/

SELECT Player, SUM(CAST(REPLACE(Salary,'$','') AS DOUBLE)) as Total_Earned_USD
FROM salaries
GROUP BY PLAYER
ORDER BY Total_Earned_USD DESC;

 * sqlite:///nbastats.db
Done.


Player,Total_Earned_USD
Kobe Bryant,328238062.0
Lebron James,237020472.0
Michael Jordan,93285000.0


In [6]:
%%sql
/*Find the average of all the salaries*/

SELECT AVG(CAST(REPLACE(Salary,'$','') AS DOUBLE)) as average_salary_USD
FROM salaries;

 * sqlite:///nbastats.db
Done.


average_salary_USD
13170870.68


In [7]:
%%sql
/*Find all salaries that are less than the average salaries*/

SELECT Player,Season,Salary FROM salaries
WHERE CAST(REPLACE(Salary,'$','') AS DOUBLE) < (SELECT AVG(CAST(REPLACE(Salary,'$','') AS DOUBLE)) 
                                                FROM salaries)


 * sqlite:///nbastats.db
Done.


Player,Season,Salary
Lebron James,2003-04,$4018920
Lebron James,2004-05,$4320360
Lebron James,2005-06,$4621800
Lebron James,2006-07,$5828090
Lebron James,2007-08,$13041250
Michael Jordan,1984-85,$550000
Michael Jordan,1985-86,$630000
Michael Jordan,1987-88,$845000
Michael Jordan,1988-89,$2000000
Michael Jordan,1989-90,$2500000


In [8]:
%%sql
/*What is the average earned by each player*/

SELECT Player, COUNT(Season) as Seasons_Played, (SUM(CAST(REPLACE(Salary,'$','') AS DOUBLE))/COUNT(Season)) 
AS Average_Earned
FROM salaries
GROUP BY Player
ORDER BY Average_Earned DESC;

 * sqlite:///nbastats.db
Done.


Player,Seasons_Played,Average_Earned
Kobe Bryant,20,16411903.1
Lebron James,15,15801364.8
Michael Jordan,15,6219000.0


In [9]:
%%sql
/*Retrieve the 10 highest scoring average in the regular season*/

SELECT Player, PTS FROM Per_game_stats
WHERE RSorPO = 'Regular Season' 
ORDER BY PTS DESC 
LIMIT 10;


 * sqlite:///nbastats.db
Done.


Player,PTS
Michael Jordan,37.1
Kobe Bryant,35.4
Michael Jordan,35
Michael Jordan,33.6
Michael Jordan,32.6
Michael Jordan,32.5
Kobe Bryant,31.6
Michael Jordan,31.5
Lebron James,31.4
Michael Jordan,30.4


In [10]:
%%sql 
/*Which player is shown the most in the 10 highest scoring average in the regular season*/

SELECT Player, COUNT(PTS) as Pts_in_top10 FROM (SELECT Player, PTS FROM Per_game_stats 
     WHERE RSorPO = 'Regular Season' 
     ORDER BY PTS DESC 
     LIMIT 10)
GROUP BY PLAYER
ORDER BY Pts_in_top10 DESC;


 * sqlite:///nbastats.db
Done.


Player,Pts_in_top10
Michael Jordan,7
Kobe Bryant,2
Lebron James,1


In [11]:
%%sql
/*Find Michael Jordans highest box plus-minus(BPM)
and compare it to his stats*/

SELECT ADS.BPM,PGS.* FROM per_game_stats PGS,advanced_stats ADS 
WHERE 
    PGS.Season = ADS.Season 
    AND PGS.Player = ADS.Player
    AND PGS.RSorPO = ADS.RSorPO
    AND ADS.BPM = (
                   SELECT MAX(BPM) FROM advanced_stats
                   WHERE Player = 'Michael Jordan' AND RSorPO = 'Regular Season');


 * sqlite:///nbastats.db
Done.


BPM,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,RSorPO
12.6,1988-89,25,CHI,NBA,SG,81,81,40.2,11.9,22.2,0.54,0.3,1.2,0.25,11.6,21.0,0.553,0.546,8.3,9.8,0.85,1.8,6.2,8.0,8.0,2.9,0.8,3.6,3.0,32.5,Michael Jordan,Regular Season


In [12]:
%%sql
/*How much did Michael Jordan make at his highest BPM*/

SELECT S.Season,S.Player, S.Salary, ADS.BPM FROM salaries S, advanced_stats ADS
WHERE
    S.Season = ADS.Season
    AND ADS.Player = S.Player
    AND ADS.RSorPO = 'Regular Season'
    AND ADS.BPM = (
                   SELECT MAX(BPM) FROM advanced_stats
                   WHERE Player = 'Michael Jordan' AND RSorPO = 'Regular Season');


 * sqlite:///nbastats.db
Done.


Season,Player,Salary,BPM
1988-89,Michael Jordan,$2000000,12.6


In [13]:
%%sql
/*Find Kobe Bryants highest box plus-minus(BPM)
and compare it to his stats*/

SELECT ADS.BPM,PGS.* FROM per_game_stats PGS,advanced_stats ADS 
WHERE 
    PGS.Season = ADS.Season 
    AND PGS.Player = ADS.Player
    AND PGS.RSorPO = ADS.RSorPO
    AND ADS.BPM = (
                   SELECT MAX(BPM) FROM advanced_stats
                   WHERE Player = 'Kobe Bryant' AND RSorPO = 'Regular Season');

 * sqlite:///nbastats.db
Done.


BPM,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,RSorPO
6.4,2002-03,24,LAL,NBA,SG,82,82,41.5,10.6,23.5,0.45,1.5,4,0.38,9.1,19.5,0.465,0.483,7.3,8.7,0.84,1.3,5.6,6.9,5.9,2.2,0.8,3.5,2.7,30,Kobe Bryant,Regular Season


In [14]:
%%sql
/*How much did Kobe Bryant make at his highest BPM*/

SELECT S.Season,S.Player, S.Salary, ADS.BPM FROM salaries S, advanced_stats ADS
WHERE
    S.Season = ADS.Season
    AND ADS.Player = S.Player
    AND ADS.RSorPO = 'Regular Season'
    AND ADS.BPM = (
                   SELECT MAX(BPM) FROM advanced_stats
                   WHERE Player = 'Kobe Bryant' AND RSorPO = 'Regular Season');


 * sqlite:///nbastats.db
Done.


Season,Player,Salary,BPM
2002-03,Kobe Bryant,$12375000,6.4


In [15]:
%%sql
/*Find Lebron James highest box plus-minus(BPM)
and compare it to his stats*/

SELECT ADS.BPM,PGS.* FROM per_game_stats PGS,advanced_stats ADS 
WHERE 
    PGS.Season = ADS.Season 
    AND PGS.Player = ADS.Player
    AND PGS.RSorPO = ADS.RSorPO
    AND ADS.BPM = (
                   SELECT MAX(BPM) FROM advanced_stats
                   WHERE Player = 'Lebron James' AND RSorPO = 'Regular Season');

 * sqlite:///nbastats.db
Done.


BPM,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,RSorPO
13.0,2008-09,24,CLE,NBA,SF,81,81,37.7,9.7,19.9,0.49,1.6,4.7,0.34,8.1,15.2,0.535,0.53,7.3,9.4,0.78,1.3,6.3,7.6,7.2,1.7,1.1,3,1.7,28.4,Lebron James,Regular Season


In [16]:
%%sql
/*How much did Lebron James make at his highest BPM using JOIN*/

SELECT S.Season,S.Player, S.Salary, ADS.BPM FROM salaries S JOIN advanced_stats ADS
ON S.Season = ADS.Season AND ADS.Player = S.Player
WHERE ADS.RSorPO = 'Regular Season'
      AND ADS.BPM = (
          SELECT MAX(BPM) FROM advanced_stats WHERE Player = 'Lebron James' 
          AND RSorPO = 'Regular Season');

 * sqlite:///nbastats.db
Done.


Season,Player,Salary,BPM
2008-09,Lebron James,$14410581,13.0
